![ecommerce_analytics-1224x532](ecommerce_analytics-1224x532.jpg)


As a Data Engineer at an electronics e-commerce company, Voltmart, I have been requested by a peer Machine Learning team to clean the data containing the information about orders made last year. They are planning to further use this cleaned data to build a demand forecasting model. To achieve this, they have shared their requirements regarding the desired output table format.

An analyst shared a parquet file called `"orders_data.parquet"` for you to clean and preprocess. 

We can see the dataset schema below along with the **cleaning requirements**:

## `orders_data.parquet`

| column | data type | description | cleaning requirements | 
|--------|-----------|-------------|-----------------------|
| `order_date` | `timestamp` | Date and time when the order was made | _Modify: Remove orders placed between 12am and 5am (inclusive); convert from timestamp to date_ |
| `time_of_day` | `string` | Period of the day when the order was made | _New column containing (lower bound inclusive, upper bound exclusive): "morning" for orders placed 5-12am, "afternoon" for orders placed 12-6pm, and "evening" for 6-12pm_ |
| `order_id` | `long` | Order ID | _N/A_ |
| `product` | `string` | Name of a product ordered | _Remove rows containing "TV" as the company has stopped selling this product; ensure all values are lowercase_ |
| `product_ean` | `double` | Product ID | _N/A_ |
| `category` | `string` | Broader category of a product | _Ensure all values are lowercase_ |
| `purchase_address` | `string` | Address line where the order was made ("House Street, City, State Zipcode") | _N/A_ |
| `purchase_state` | `string` | US State of the purchase address | _New column containing: the State that the purchase was ordered from_ |
| `quantity_ordered` | `long` | Number of product units ordered | _N/A_ |
| `price_each` | `double` | Price of a product unit | _N/A_ |
| `cost_price` | `double` | Cost of production per product unit | _N/A_ |
| `turnover` | `double` | Total amount paid for a product (quantity x price) | _N/A_ |
| `margin` | `double` | Profit made by selling a product (turnover - cost) | _N/A_ |

<br>

In [1]:
from pyspark.sql import (
    SparkSession,
    types,
    functions as F,
)
# Initiate a Spark session
spark = (
    SparkSession
    .builder
    .appName('cleaning_orders_dataset_with_pyspark')
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/20 04:47:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# IMPORT DATA

# Read data from the parquet file
orders_data = spark.read.parquet('orders_data.parquet')
orders_data.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950


### 1. Add new column containing (lower bound inclusive, upper bound exclusive): "morning" for orders placed 5-12am, "afternoon" for orders placed 12-6pm, and "evening" for 6-12pm 

In [8]:
# DATA CLEANING AND PREPROCESSING

from pyspark.sql import functions as F

orders_data = (
    orders_data
    # Create a new column time_of_day
    .withColumn(
        'time_of_day',
        # When/otherwise (similar to case/when/else) statements extracting hour from timestamp
        F.when((F.hour('order_date') >= 0) & (F.hour('order_date') <= 5), 'night')
         .when((F.hour('order_date') >= 6) & (F.hour('order_date') <= 11), 'morning')
         .when((F.hour('order_date') >= 12) & (F.hour('order_date') <= 17), 'afternoon')
         .when((F.hour('order_date') >= 18) & (F.hour('order_date') <= 23), 'evening')
        # Keep the otherwise statement as None to validate whether the conditions are exhaustive
         .otherwise(None)
    )
)

orders_data.show(5, truncate=False)

+-------------------+--------+------------------------+-----------------+------------+-------------------------------------+----------------+----------+----------+--------+-------+-----------+
|order_date         |order_id|product                 |product_id       |category    |purchase_address                     |quantity_ordered|price_each|cost_price|turnover|margin |time_of_day|
+-------------------+--------+------------------------+-----------------+------------+-------------------------------------+----------------+----------+----------+--------+-------+-----------+
|2023-01-22 21:25:00|141234  |iPhone                  |5.638008983335E12|Vêtements   |944 Walnut St, Boston, MA 02215      |1               |700.0     |231.0     |700.0   |469.0  |evening    |
|2023-01-28 14:15:00|141235  |Lightning Charging Cable|5.563319511488E12|Alimentation|185 Maple St, Portland, OR 97035     |1               |14.95     |7.475     |14.95   |7.475  |afternoon  |
|2023-01-17 13:33:00|141236  |Wired

### 2. Remove orders placed between 12am and 5am (inclusive); convert from timestamp to date

In [10]:
orders_data = (
    orders_data
    # Filter "night" out from the column
    .filter(
        F.col('time_of_day') != 'night'
    )
    # Cast order_date to date as it is originally a timestamp
    .withColumn(
        'order_date',
        F.col('order_date').cast(types.DateType())
    )
)
orders_data.show(5, truncate=False)

+----------+--------+------------------------+-----------------+------------+-------------------------------------+----------------+----------+----------+--------+-------+-----------+
|order_date|order_id|product                 |product_id       |category    |purchase_address                     |quantity_ordered|price_each|cost_price|turnover|margin |time_of_day|
+----------+--------+------------------------+-----------------+------------+-------------------------------------+----------------+----------+----------+--------+-------+-----------+
|2023-01-22|141234  |iPhone                  |5.638008983335E12|Vêtements   |944 Walnut St, Boston, MA 02215      |1               |700.0     |231.0     |700.0   |469.0  |evening    |
|2023-01-28|141235  |Lightning Charging Cable|5.563319511488E12|Alimentation|185 Maple St, Portland, OR 97035     |1               |14.95     |7.475     |14.95   |7.475  |afternoon  |
|2023-01-17|141236  |Wired Headphones        |2.11397339522E12 |Vêtements   |538

### 3.Convert product and category column to lowercase and romove rows containing "TV" as the company has stopped selling this product.

In [23]:
orders_data = (
    orders_data
    # Make product and category columns lowercase
    .withColumn(
        'product',
        F.lower('product')
    )
    .withColumn(
        'category',
        F.lower('category')
    )
    # Remove rows where product column contains "tv" 
    .filter(
        ~F.col('product').contains('tv')
    )
)
orders_data.show(5, truncate=False)

+----------+--------+------------------------+-----------------+------------+-------------------------------------+----------------+----------+----------+--------+-------+-----------+
|order_date|order_id|product                 |product_id       |category    |purchase_address                     |quantity_ordered|price_each|cost_price|turnover|margin |time_of_day|
+----------+--------+------------------------+-----------------+------------+-------------------------------------+----------------+----------+----------+--------+-------+-----------+
|2023-01-22|141234  |iphone                  |5.638008983335E12|vêtements   |944 Walnut St, Boston, MA 02215      |1               |700.0     |231.0     |700.0   |469.0  |evening    |
|2023-01-28|141235  |lightning charging cable|5.563319511488E12|alimentation|185 Maple St, Portland, OR 97035     |1               |14.95     |7.475     |14.95   |7.475  |afternoon  |
|2023-01-17|141236  |wired headphones        |2.11397339522E12 |vêtements   |538

### 4.Create new column containing: the State that the purchase was ordered from

In [24]:

orders_data = (
    orders_data
    # Split the purchase address by space (" ")
    .withColumn(
        'address_split',
        F.split('purchase_address', ' ')
    )
    # The state abbreviation is always at the 2nd last position
    .withColumn(
        'purchase_state',
        F.col('address_split').getItem(F.size('address_split') - 2)
    )
    # Dropping address_split columns as it is a temporary technical column
    .drop('address_split')
)
orders_data.show(5, truncate=False)


+----------+--------+------------------------+-----------------+------------+-------------------------------------+----------------+----------+----------+--------+-------+-----------+--------------+
|order_date|order_id|product                 |product_id       |category    |purchase_address                     |quantity_ordered|price_each|cost_price|turnover|margin |time_of_day|purchase_state|
+----------+--------+------------------------+-----------------+------------+-------------------------------------+----------------+----------+----------+--------+-------+-----------+--------------+
|2023-01-22|141234  |iphone                  |5.638008983335E12|vêtements   |944 Walnut St, Boston, MA 02215      |1               |700.0     |231.0     |700.0   |469.0  |evening    |MA            |
|2023-01-28|141235  |lightning charging cable|5.563319511488E12|alimentation|185 Maple St, Portland, OR 97035     |1               |14.95     |7.475     |14.95   |7.475  |afternoon  |OR            |
|2023

In [28]:
# Use distinct and count to calculate the number of unique values
n_states = (
    orders_data
    .select('purchase_state')
    .distinct()
    .count()
)
print(n_states)

8


In [26]:
# EXPORT

# Export the resulting table to parquet format with the new name
(
    orders_data
    .write
    .parquet(
        'orders_data_clean.parquet',
        mode='overwrite',
    )
)